In [3]:
import json
import random
import string

# User Input
################################################

working_dir = '/Users/julianchu/Desktop/new_series'

project_name = 'drama4'

characters = {
    "Claire": "A creative and determined woman in her late 20s with fair skin, wavy auburn hair often tied back in a loose ponytail, and expressive green eyes. She is usually dressed in a floral apron dusted with flour over casual sweaters and jeans, pairing practicality with charm. Claire's warm smile belies her frazzled energy as she juggles her struggling business and personal dilemmas.",
    "Ryan": "A laid-back and supportive man in his early 30s with a lanky but athletic build. He has messy dark brown hair, warm hazel eyes, and a perpetual five o'clock shadow. Ryan's casual wardrobe of flannels, jeans, and sneakers reflects his easygoing personality. He often carries a thermos of coffee or a donut, ready with a quick-witted remark and a helping hand.",
}

style = "A Japanese shonen anime shot"

################################################


trigger_words = {}
for character in characters:
    # Generate random 4-character string
    random_str = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))
    trigger_word = f"{random_str}_{character.lower()}"
    trigger_words[character] = trigger_word
    print(f"Trigger word for {character}: {trigger_word}")


# Create character config dictionary with trigger words
character_config = {
    "project_name": project_name,
    "characters": {
        trigger_words[character]: {
            "name": character,
            "description": description,
            "trigger_word": trigger_words[character], 
            "finetune": True # set to false in the config if you don't want to finetune that character
        } for character, description in characters.items()
    },
    "style": style
}

# Check if config file already exists
import os
if os.path.exists(f'{working_dir}/character_config.json'):
    raise FileExistsError(f"Config file already exists at {working_dir}/character_config.json")



with open(f'{working_dir}/character_config.json', 'w') as f:
    json.dump(character_config, f, indent=2)




In [4]:
config_file = 'character_config.json'

# Load character config
with open(f'{working_dir}/{config_file}', 'r') as f:
    character_config = json.load(f)
character_config

{'characters': {'mdg4_claire': {'name': 'Claire',
   'description': "A creative and determined woman in her late 20s with fair skin, wavy auburn hair often tied back in a loose ponytail, and expressive green eyes. She is usually dressed in a floral apron dusted with flour over casual sweaters and jeans, pairing practicality with charm. Claire's warm smile belies her frazzled energy as she juggles her struggling business and personal dilemmas.",
   'trigger_word': 'mdg4_claire',
   'finetune': True},
  'ikzg_ryan': {'name': 'Ryan',
   'description': "A laid-back and supportive man in his early 30s with a lanky but athletic build. He has messy dark brown hair, warm hazel eyes, and a perpetual five o'clock shadow. Ryan's casual wardrobe of flannels, jeans, and sneakers reflects his easygoing personality. He often carries a thermos of coffee or a donut, ready with a quick-witted remark and a helping hand.",
   'trigger_word': 'ikzg_ryan',
   'finetune': False,
   'finetuned_model': 'julian

In [31]:
from character_creation import generate_character_images

for character in character_config['characters']:
    generate_character_images(character, character_config['characters'][character], character_config['style'], working_dir) # prompt will be {style} of {description}

Generating image for mdg4_claire...
mdg4_claire image generated and saved as /Users/julianchu/Desktop/new_series/characters/mdg4_claire.jpg

Generating image for ikzg_ryan...
ikzg_ryan image generated and saved as /Users/julianchu/Desktop/new_series/characters/ikzg_ryan.jpg



In [32]:
from character_creation import generate_character_variations

for character in character_config['characters']:
    if character_config['characters'][character]['finetune']:
        generate_character_variations(character, character_config['style'], working_dir, number_of_outputs=2)


Variation 1 saved as /Users/julianchu/Desktop/new_series/characters/mdg4_claire/variation_1.jpg
Variation 2 saved as /Users/julianchu/Desktop/new_series/characters/mdg4_claire/variation_2.jpg
Generated variations for mdg4_claire



In [34]:
from character_creation import process_variations_for_finetuning

process_variations_for_finetuning(character_config, working_dir, zip_dir=True)


Processed variation 1 for mdg4_claire: mdg4_claire is looking over her shoulder and smiling softly while holding a large piece of pink crystal or mineral.
Zipped /Users/julianchu/Desktop/new_series/characters/mdg4_claire to /Users/julianchu/Desktop/new_series/characters/mdg4_claire.zip
Processed variation 2 for mdg4_claire: mdg4_claire is looking forward with a slight smile, holding up an object in one hand and touching her hair with the other.
Zipped /Users/julianchu/Desktop/new_series/characters/mdg4_claire to /Users/julianchu/Desktop/new_series/characters/mdg4_claire.zip
Skipping ikzg_ryan as finetune is False

All variations processed with corresponding caption files


In [ ]:
from character_creation import finetune_character

# Finetune characters and save model paths
for character in character_config['characters']:
    if character_config['characters'][character]['finetune']:
        # Finetune the character and get model path
        model_path = finetune_character(working_dir, character)
        
        # Save the model path in the config
        character_config['characters'][character]['finetuned_model'] = model_path

# Save updated config back to file
with open(f'{working_dir}/{config_file}', 'w') as f:
    json.dump(character_config, f, indent=2)
